# Study Summary Report
> Author: Clayton Herbst

## Business Needs Statement
CRO need to be able to review the state of a study on demand, following the progression of study participants over time using key metrics captured within the study.

## Task Description
Generate a report that reflects the current data within the study, collating all ECG, Vital Signs, PK data on a per tab basis ensuring the links to study participants, forms, visits and timepoints remain. Data must accurately reflect the data stored in the study database.

In [1]:
from openpyxl import Workbook, load_workbook
import json
import pprint
import datetime
import os
import re
from matplotlib import pyplot
from src import FileReader, ParseData, FileWriter

%matplotlib inline

## Setup Output Folder

Create the __output folder__ in which all output files will be placed.

In [2]:
# Set the output folder location
outputfolder = './test/output'

In [3]:
if not os.path.exists(outputfolder):
    os.makedirs(outputfolder, 0o755) # owner execution 

View contents of folder:

In [4]:
%ls -l ./test

total 18632
-rw-r--r--@ 1 herbsca  staff  9531070 19 May 13:21 browse_data.xlsx
-rw-r--r--@ 1 herbsca  staff     3742 19 May 17:04 config.json
drwxr-xr-x  5 herbsca  staff      160 19 May 17:04 output/
-rw-r--r--@ 1 herbsca  staff      165 19 May 13:21 ~$browse_data.xlsx


## Define Utility Functions
These functions generate the format of the output file. They are supplied with a _dictionary variable_ (`contents`) and then convert this information structure into the output file format (csv -> columns, json -> key-value pairs etc).

In [5]:
def writeFile(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(pprint.pformat(contents))
    print('Text file write complete.')

def writeJSON(filename: str, contents: object):
    # Create function to allow for file creation in output.
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        outputfile.write(json.dumps(contents))
    print('JSON file write complete.')

def writeErrorCSV(filename: str, contents: object):
    path = os.path.join(outputfolder, filename)
    with open(path, 'w') as outputfile:
        fieldnames = ['subjectid', 'type', 'collection', 'visit', 'panel', 'sourceDate', 'sourceTime', 'comparisonDate', 'comparisonTime', 'message']
        writer = csv.DictWriter(outputfile, fieldnames=fieldnames)
        writer.writeheader() # place headers in csv file.
        for key in contents:
            if key == 'errors' or  key  == 'total':
                    continue
            for data in contents[key]:
                # cycle through list of errors
                sourcedate = '-'
                sourcetime = '-'
                comparisondate = '-'
                comparisontime = '-'
                msg =  ''
                if 'date' in data and data['date']['error']:
                    sourcedate = data['date']['source']
                    comparisondate = data['date']['comparison']
                if 'time' in data and data['time']['error']:
                    sourcetime = data['time']['source']
                    comparisontime = data['time']['comparison']
                if 'msg' in data:
                    msg = data['msg']

                row = {
                    'subjectid': data['subject'],
                    'visit': data['visit'],
                    'type': data['type'],
                    'collection': data['collection'],
                    'panel': data['panel'],
                    'sourceDate': sourcedate,
                    'sourceTime': sourcetime,
                    'comparisonDate': comparisondate,
                    'comparisonTime': comparisontime,
                    'message': msg
                }
                writer.writerow(row) # write to file
    print('CSV file write complete.')

## Parse Config File

__EDIT HERE:__

In [6]:
config_file_path = './test/config.json'

Parse config file and establish relationships.

In [7]:
config_dict = dict()

with open(config_file_path) as jsonfile:
    config_dict = json.load(jsonfile)

pprint.pp(config_dict)

{'tabs': ['ECG',
          'Vital Signs',
          'DSST',
          'Neurological',
          'Physical Exam',
          'Dose Administration',
          'Adverse Events'],
 'map': {'ECG': {'_formregex': 'ecg',
                 '_col': ['date',
                          'time',
                          'heart',
                          'rr',
                          'pr',
                          'qrsd',
                          'qt',
                          'qtcf',
                          'asssessment'],
                 '_medrio_order': ['date',
                                   'time',
                                   'heart',
                                   'rr',
                                   'pr',
                                   'qrsd',
                                   'qt',
                                   'qtcf',
                                   'assessment'],
                 '_colregex': {'date': 'dat',
                               'time': 'tim

## Process Workbook

__EDIT HERE:__ Specify where the input file can be found.

In [8]:
input_file_path = './test/browse_data.xlsx'

Read the xlsx file and create a searchable dictionary variable.

In [9]:
# Create File Reader Object
filereader = FileReader(input_file_path)
ignore_col_before = 4 # number of rows to ignore --> i.e row 1 to 5

# Get File Contents
headers = filereader.getSheetHeaders() # all medrio variables etc
ws = filereader.getWorksheet() # worksheet for the file read

# Create File Processing Object
parser = ParseData(headers, config_dict.get('map', None)) # pd = processdata -- allows data to be processed

# Parse Contents
row_id = 0 # keep track of number of rows looked at
for row in ws.rows:
    data = dict()
    row_id += 1 # increment the row
    if row_id == 1:
        continue # skip the xlsx headers
    
    # -- TESTING --
    if row_id > 200:
        break 
    general = parser.process_generalised_cells(row)
    print(general.get('Form Name', None))
    holder = parser.generate_form_type(general.get('Form Name', None)) # get the form identifier
    
    formtype = holder[0]
    is_triplicate = holder[1]
    
        
    if formtype is None:
        continue
    else:
        print(formtype)
    
    if is_triplicate: # is triplicate
        data = parser.process_triplicate_from_config(row, formtype)
        if data is None:
            continue
    else: # not triplicate
        data = parser.process_from_config(row, formtype, ignore_col_before)
        print(data)
        if data is None:
            continue
    
    # -- ADD TO OVERALL DICTIONARY --
    #pprint.pp(general)
    #print(data)
    parser.add_row({**general, **data}, formtype) # use the formtype as the output file tab name
    pprint.pp({**general, **data})

# -- FORMAT THE DATA --
parser.format_data()

Neurological Exam
Neurological
{'assessment': 'Other, please specify', 'significance': 'Not Done'}
{'Subject ID': 'S003-11001',
 'Form Name': 'Neurological Exam',
 'Group': 'SAD - COH 1',
 'Visit': 'Day 7',
 'assessment': 'Other, please specify',
 'significance': 'Not Done'}
Neurological Exam
Neurological
{'assessment': 'Fine touch on foot / proprioception (large toe)', 'significance': 'Normal'}
{'Subject ID': 'S003-11001',
 'Form Name': 'Neurological Exam',
 'Group': 'SAD - COH 1',
 'Visit': 'Screening',
 'assessment': 'Fine touch on foot / proprioception (large toe)',
 'significance': 'Normal'}
Complete Physical Exam
Physical Exam
{'system': 'Abdomen', 'result': 'Normal'}
{'Subject ID': 'S003-11001',
 'Form Name': 'Complete Physical Exam',
 'Group': 'SAD - COH 1',
 'Visit': 'Day -1',
 'system': 'Abdomen',
 'result': 'Normal'}
Complete Physical Exam
Physical Exam
{'system': 'Lymph Nodes', 'result': 'Normal'}
{'Subject ID': 'S003-11001',
 'Form Name': 'Complete Physical Exam',
 'Group'

In [10]:
pprint.pp(parser.get_data())
writeJSON('temp_output.json', parser.get_data())

{'Neurological': [{'Subject ID': 'S003-11001',
                   'Form Name': 'Neurological Exam',
                   'Group': 'SAD - COH 1',
                   'Visit': 'Day 7',
                   'Other, please specify': 'Not Done',
                   'Flexion & Extension of the leg at the Knee and foot (ankle)': 'Normal',
                   'date': '04/23/2020',
                   'time': '11:54',
                   'Finger-nose Pointing / Heel-knee-toe Movement': 'Normal',
                   'find': 'Trice/biceps 1+\nPatella/ankle 2+',
                   'Reflexes of Elbow (biceps, triceps), Knee & Ankle': 'Normal',
                   'Gait Assessment': 'Normal',
                   'Eye Movement / Nystagmus': 'Normal',
                   'Fine touch on foot / proprioception (large toe)': 'Normal'},
                  {'Subject ID': 'S003-11001',
                   'Form Name': 'Neurological Exam',
                   'Group': 'SAD - COH 1',
                   'Visit': 'Screening',
 

## Output Results to XLSX File

In [11]:
output_filename = 'output'

tabs = config_dict.get('tabs', None) # Fetch tabs to be included from config file

# Create file writer object
writer = FileWriter(outputfolder, output_filename, tabs, parser.get_data())

for tab in parser.get_data().keys():
    writer.write_worksheet(tab)

writer.close() # saves the file to disk